# MNIST - Python

## 1.MNIST 데이터 로딩

In [3]:
# coding: utf-8
import urllib.request
import os.path
import gzip
import pickle
import os
import numpy as np

url_base = 'http://yann.lecun.com/exdb/mnist/'
key_file = {
    'train_img':'train-images-idx3-ubyte.gz',
    'train_label':'train-labels-idx1-ubyte.gz',
    'test_img':'t10k-images-idx3-ubyte.gz',
    'test_label':'t10k-labels-idx1-ubyte.gz'
}

dataset_dir = os.path.dirname("/Users/yhhan/git/aiclass/0.Professor/3.VanillaNN/MNIST_data/.")
save_file = dataset_dir + "/mnist.pkl"

train_num = 60000
test_num = 10000
img_dim = (1, 28, 28)
img_size = 784

def _download(file_name):
    file_path = dataset_dir + "/" + file_name
    print(file_path)
    if os.path.exists(file_path):
        return

    print("Downloading " + file_name + " ... ")
    urllib.request.urlretrieve(url_base + file_name, file_path)
    print("Done")
    
def download_mnist():
    for v in key_file.values():
       _download(v)
        
def _load_label(file_name):
    file_path = dataset_dir + "/" + file_name
    
    print("Converting " + file_name + " to NumPy Array ...")
    with gzip.open(file_path, 'rb') as f:
            labels = np.frombuffer(f.read(), np.uint8, offset=8)
    print("Done")
    
    return labels

def _load_img(file_name):
    file_path = dataset_dir + "/" + file_name
    
    print("Converting " + file_name + " to NumPy Array ...")    
    with gzip.open(file_path, 'rb') as f:
            data = np.frombuffer(f.read(), np.uint8, offset=16)
    data = data.reshape(-1, img_size)
    print("Done")
    
    return data
    
def _convert_numpy():
    dataset = {}
    dataset['train_img'] =  _load_img(key_file['train_img'])
    dataset['train_label'] = _load_label(key_file['train_label'])
    dataset['test_img'] = _load_img(key_file['test_img'])
    dataset['test_label'] = _load_label(key_file['test_label'])

    dataset['validation_img'] = dataset['train_img'][55000:]
    dataset['validation_label'] = dataset['train_label'][55000:]
    dataset['train_img'] =  dataset['train_img'][:55000]
    dataset['train_label'] = dataset['train_label'][:55000]
    return dataset

def init_mnist():
    download_mnist()
    dataset = _convert_numpy()
    print("Creating pickle file ...")
    with open(save_file, 'wb') as f:
        pickle.dump(dataset, f, -1)
    print("Done!")

def _change_one_hot_label(X):
    T = np.zeros((X.size, 10))
    for idx, row in enumerate(T):
        row[X[idx]] = 1
        
    return T
    

def load_mnist(normalize=True, flatten=True, one_hot_label=False):
    if not os.path.exists(save_file):
        init_mnist()
        
    with open(save_file, 'rb') as f:
        dataset = pickle.load(f)
    
    if normalize:
        for key in ('train_img', 'validation_img', 'test_img'):
            dataset[key] = dataset[key].astype(np.float32)
            dataset[key] /= 255.0
            
    if one_hot_label:
        dataset['train_label'] = _change_one_hot_label(dataset['train_label'])
        dataset['validation_label'] = _change_one_hot_label(dataset['test_label'])
        dataset['test_label'] = _change_one_hot_label(dataset['test_label'])
    
    if not flatten:
         for key in ('train_img', 'validation_img', 'test_img'):
            dataset[key] = dataset[key].reshape(-1, 1, 28, 28)

    return (dataset['train_img'], dataset['train_label']), (dataset['validation_img'], dataset['validation_label']), (dataset['test_img'], dataset['test_label']) 


if __name__ == '__main__':
    init_mnist()


/Users/yhhan/git/aiclass/0.Professor/3.VanillaNN/MNIST_data/train-images-idx3-ubyte.gz
Done
/Users/yhhan/git/aiclass/0.Professor/3.VanillaNN/MNIST_data/train-labels-idx1-ubyte.gz
Done
/Users/yhhan/git/aiclass/0.Professor/3.VanillaNN/MNIST_data/t10k-images-idx3-ubyte.gz
Done
/Users/yhhan/git/aiclass/0.Professor/3.VanillaNN/MNIST_data/t10k-labels-idx1-ubyte.gz
Done
Converting train-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting train-labels-idx1-ubyte.gz to NumPy Array ...
Done
Converting t10k-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting t10k-labels-idx1-ubyte.gz to NumPy Array ...
Done
Creating pickle file ...
Done!


## 2. 2층 신경망을 이용한 MNIST 학습 및 모델 검증

In [15]:
def step_function(x):
    return np.array(x > 0, dtype=np.int)

def softmax(x):
    c = np.max(x)
    exp_x = np.exp(x-c)
    sum_exp_x = np.sum(exp_x)
    y = exp_x / sum_exp_x
    return y

class TwoLayerNet:
    def __init__(self, input_size, hidden_layer1_size, hidden_layer2_size, output_size, weight_init_std=0.01):
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_layer1_size)
        self.params['b1'] = np.zeros(hidden_layer1_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_layer1_size, hidden_layer2_size)
        self.params['b2'] = np.zeros(hidden_layer2_size)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_layer2_size, output_size)
        self.params['b3'] = np.zeros(output_size)
        print("W1-shape: {0}, b1-shape: {1}, W2-shape: {2}, b2-shape: {3}, W3-shape: {4}, b3-shape: {5}".format(
            self.params['W1'].shape,
            self.params['b1'].shape,
            self.params['W2'].shape,
            self.params['b2'].shape,
            self.params['W3'].shape,
            self.params['b3'].shape
        ))

    def predict(self, x):
        W1, W2, W3 = self.params['W1'], self.params['W2'], self.params['W3']
        b1, b2, b3 = self.params['b1'], self.params['b2'], self.params['b3']
    
        a1 = np.dot(x, W1) + b1
        z1 = step_function(a1)
        a2 = np.dot(z1, W2) + b2
        z2 = step_function(a2)
        a3 = np.dot(z2, W3) + b3
        y = softmax(a3)
        
        return y

    def cross_entropy_error(self, x, t):
        y = self.predict(x)
        if y.ndim == 1:
            t = t.reshape(1, t.size)
            y = y.reshape(1, y.size)

        if t.size == y.size:
            t = t.argmax(axis=1)

        batch_size = y.shape[0]
        return -np.sum(np.log(y[np.arange(batch_size), t])) / batch_size
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy

    def numerical_derivative(self, params, x, z_target):
        delta = 1e-4 # 0.0001
        grad = np.zeros_like(params)
       
        it = np.nditer(params, flags=['multi_index'], op_flags=['readwrite'])
        while not it.finished:
            idx = it.multi_index
            temp_val = params[idx]

            #f(x + delta) 계산
            params[idx] = params[idx] + delta
            fxh1 = self.cross_entropy_error(x, z_target)
            
            #f(x - delta) 계산
            params[idx] = params[idx] - delta
            fxh2 = self.cross_entropy_error(x, z_target)
            
            #f(x + delta) - f(x - delta) / 2 * delta 계산
            grad[idx] = (fxh1 - fxh2) / (2 * delta)
            params[idx] = temp_val
            it.iternext()
        return grad
   
    def learning(self, learning_rate, x_batch, t_batch):
        for key in ('W1', 'b1', 'W2', 'b2', 'W3', 'b3'):
            print(key, end=', ', flush=True)
            grad = self.numerical_derivative(self.params[key], x_batch, t_batch)
            self.params[key] = self.params[key] - learning_rate * grad

In [17]:
import math

(img_train, label_train), (img_validation, label_validation), (img_test, label_test) = load_mnist(flatten=True, normalize=False)

network = TwoLayerNet(input_size=784, hidden_layer1_size=512, hidden_layer2_size=512, output_size=10)

num_epochs = 3000
train_size = img_train.shape[0]
batch_size = 1000
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
validation_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(num_epochs):
    num_batch = math.ceil(train_size / batch_size)
    j = 0
    for j in range(num_batch):
        print(j * batch_size, end=', ', flush=True)
        x_batch = img_train[j * batch_size : j * batch_size + batch_size]
        t_batch = label_train[j * batch_size : j * batch_size + batch_size]
        network.learning(learning_rate, x_batch, t_batch)
    
    error = network.cross_entropy_error(img_train, label_train)
    train_loss_list.append(error)
    
    train_acc = network.accuracy(img_train, label_train)
    train_acc_list.append(train_acc)

    validation_acc = network.accuracy(img_validation, label_validation)
    validation_acc_list.append(validation_acc)

    print("Epoch: {0:3d}, Train accuracy: {1:7.5}, validation accuracy: {2:7.5f}".format(
        i,
        train_acc,
        validation_acc
    ))

W1-shape: (784, 16), b1-shape: (16,), W2-shape: (16, 16), b2-shape: (16,), W3-shape: (16, 10), b3-shape: (10,)
0, W1, 

KeyboardInterrupt: 